# HyperbolicTSNE

This notebook illustrates the usage of the HyperbolicTSNE library. Specifically, we load a subset of the MNIST dataset and embed it in hyperbolic space using the accelerated version of hyperbolic tsne. Finally, we save the embedding result as an image.

## Setup

First, we import the packages we will use and set important paths. Note that `hyperbolicTSNE.util` and `hyperbolicTSNE.visualization` contain useful functions for reading, processing and exporting embeddings. This requires that hyperbolicTSNE has been set up as detailed in the main readme of the repository. 

In [1]:
import os
import traceback

from hyperbolicTSNE.util import find_last_embedding
from hyperbolicTSNE.visualization import plot_poincare, animate
from hyperbolicTSNE import load_data, Datasets, SequentialOptimizer, initialization, HyperbolicTSNE

Please note that `empty_sequence` uses the KL divergence with Barnes-Hut approximation (angle=0.5) by default.


We assume that there is a top-level folder `datasets` that holds the MNIST data set. Refer to the main readme of the repository for where to find the data sets used in this repository.

In [2]:
data_home = "datasets"
log_path = "temp/poincare/"  # path for saving embedding snapshots

## Configure

HyperbolicTSNE follows a similar API to other t-SNE libraries like OpenTSNE and sklearn. The configuration process consists of loading the data to embed and defining the settings of the embedder. We create a dict with parameters manually to demonstrate all the customization options. Nevertheless, `hyperbolicTSNE.hyperbolicTSNE` provides parameter templates to start with.

In [3]:
only_animate = False
seed = 42
dataset = Datasets.C_ELEGANS  # the Datasets handler provides access to several data sets used throughout the repository
num_points = 10000  # we use a subset for demonstration purposes, full MNIST has N=70000
perp = 30  # we use a perplexity of 30 in this example

dataX, dataLabels, D, V, _ = load_data(
    dataset, 
    data_home=data_home, 
    random_state=seed, 
    to_return="X_labels_D_V",
    hd_params={"perplexity": perp}, 
    sample=num_points, 
    knn_method="hnswlib"  # we use an approximation of high-dimensional neighbors to speed up computations
)

In [4]:
exaggeration_factor = 12  # Just like regular t-SNE, we use early exaggeration with a factor of 12
learning_rate = (dataX.shape[0] * 1) / (exaggeration_factor * 1000)  # We adjust the learning rate to the hyperbolic setting
ex_iterations = 250  # The embedder is to execute 250 iterations of early exaggeration, ...
main_iterations = 750  # ... followed by 750 iterations of non-exaggerated gradient descent.

opt_config = dict(
    learning_rate_ex=learning_rate,  # learning rate during exaggeration
    learning_rate_main=learning_rate,  # learning rate main optimization 
    exaggeration=exaggeration_factor, 
    exaggeration_its=ex_iterations, 
    gradientDescent_its=main_iterations, 
    vanilla=False,  # if vanilla is set to true, regular gradient descent without any modifications is performed; for  vanilla set to false, the optimization makes use of momentum and gains
    momentum_ex=0.5,  # Set momentum during early exaggeration to 0.5
    momentum=0.8,  # Set momentum during non-exaggerated gradient descent to 0.8
    exact=False,  # To use the quad tree for acceleration (like Barnes-Hut in the Euclidean setting) or to evaluate the gradient exactly
    area_split=False,  # To build or not build the polar quad tree based on equal area splitting or - alternatively - on equal length splitting
    n_iter_check=10,  # Needed for early stopping criterion
    size_tol=0.999  # Size of the embedding to be used as early stopping criterion
)

opt_params = SequentialOptimizer.sequence_poincare(**opt_config)

# Start: configure logging
logging_dict = {
    "log_path": log_path
}
opt_params["logging_dict"] = logging_dict

log_path = opt_params["logging_dict"]["log_path"]
# Delete old log path
if os.path.exists(log_path) and not only_animate:
    import shutil
    shutil.rmtree(log_path)
# End: logging

print(f"config: {opt_config}")

Please note that `empty_sequence` uses the KL divergence with Barnes-Hut approximation (angle=0.5) by default.
config: {'learning_rate_ex': 0.8333333333333334, 'learning_rate_main': 0.8333333333333334, 'exaggeration': 12, 'exaggeration_its': 250, 'gradientDescent_its': 750, 'vanilla': False, 'momentum_ex': 0.5, 'momentum': 0.8, 'exact': False, 'area_split': False, 'n_iter_check': 10, 'size_tol': 0.999}


## Run HyperbolicTSNE

Embedding the high dimensional data consists of three steps:
- Initializating the embedding
- Initializing the embedder 
- Embedding the data

The following three cells demonstrate this process. Note that use set metric to "precomputed" because we pass the distance matrix to the `fit` method.

In [5]:
# Compute an initial embedding of the data via PCA
X_embedded = initialization(
    n_samples=dataX.shape[0],
    n_components=2,
    X=dataX,
    random_state=seed,
    method="pca"
)

In [6]:
# Initialize the embedder
htsne = HyperbolicTSNE(
    init=X_embedded, 
    n_components=2, 
    metric="precomputed", 
    verbose=True, 
    opt_method=SequentialOptimizer, 
    opt_params=opt_params
)

In [7]:
# Compute the embedding
try:
    hyperbolicEmbedding = htsne.fit_transform((D, V))
except ValueError:
    hyperbolicEmbedding = find_last_embedding(log_path)
    traceback.print_exc()

[HyperbolicTSNE] Received iterable as input. It should have len=2 and contain (D=None, V=None)
[hd_mat] Warning: There is nothing to do with given parameters. Returning given D and V
Running Gradient Descent, Verbosity: True
[gradient_descent] Warning: because of logging, the cf will be computed at every iteration


Gradient Descent error: 168.60138 grad_norm: 6.81478e-01:   2%|█▊                                                                                       | 5/250 [02:00<1:40:11, 24.54s/it]

[t-SNE] Tree: 11153186 clock ticks | [t-SNE] Tree: 14301697 clock ticks | Force computation: 181884864 clock ticks
[t-SNE] Tree: 12079877 clock ticks | Force computation: 161160291 clock ticks
[t-SNE] Tree: 11653510 clock ticks | Force computation: 164425666 clock ticks
[t-SNE] Tree: 12993119 clock ticks | Force computation: 178847133 clock ticks
[t-SNE] Tree: 12118380 clock ticks | Force computation: 179819432 clock ticks
[t-SNE] Tree: 14484286 clock ticks | Force computation: 177890930 clock ticks
[t-SNE] Tree: 14437356 clock ticks | Force computation: 180211625 clock ticks
[t-SNE] Tree: 14519132 clock ticks | Force computation: 180401994 clock ticks
[t-SNE] Tree: 13219466 clock ticks | Force computation: 181500007 clock ticks
[t-SNE] Tree: 13278560 clock ticks | Force computation: 181490874 clock ticks
Force computation: 160969319 clock ticks
[t-SNE] Tree: 12983633 clock ticks | Force computation: 183289092 clock ticks
[t-SNE] Tree: 8554684 clock ticks | [t-SNE] Tree: 13802083 clock

Gradient Descent error: 168.60144 grad_norm: 6.78847e-01:   4%|███▏                                                                                     | 9/250 [03:39<1:39:13, 24.70s/it]

ree: 14153184 clock ticks | [t-SNE] Tree: 14235899 clock ticks | Force computation: 200384192 clock ticks
Force computation: 198235711 clock ticks
Force computation: 193294196 clock ticks
[t-SNE] Tree: 13338018 clock ticks | Force computation: 201594113 clock ticks
[t-SNE] Tree: 12973888 clock ticks | Force computation: 200810106 clock ticks
Force computation: 191732541 clock ticks
[t-SNE] Tree: 12792340 clock ticks | Force computation: 201626298 clock ticks
[t-SNE] Tree: 13498030 clock ticks | Force computation: 201041156 clock ticks
Force computation: 187055127 clock ticks
[t-SNE] Tree: 12745752 clock ticks | [t-SNE] Tree: 13049526 clock ticks | Force computation: 203145024 clock ticks
Force computation: 199851846 clock ticks
[t-SNE] Tree: 13321132 clock ticks | [t-SNE] Tree: 13953052 clock ticks | Force computation: 207562352 clock ticks
[t-SNE] Tree: 13352231 clock ticks | [t-SNE] Tree: 13410016 clock ticks | Force computation: 207471275 clock ticks
[t-SNE] Tree: 12928462 clock tic

Gradient Descent error: 168.60146 grad_norm: 6.78413e-01:   4%|███▊                                                                                    | 11/250 [04:54<1:46:39, 26.78s/it]


KeyboardInterrupt: 

## Exporting and visualization

After running the embedding process, the embeddings arrays are saved to the `log_path`. We can use this information to visualize the embeddings using utility functions defined in `hyperbolicTSNE.visualization` as shown below.

In [ ]:
# Create a rendering of the embedding and save it to a file
if not os.path.exists("results"):
    os.mkdir("results")
fig = plot_poincare(hyperbolicEmbedding, dataLabels)
fig.savefig(f"results/{dataset.name}.png")

In [ ]:
# This renders a GIF animation of the embedding process. If FFMPEG is installed, the command also supports .mp4 as file ending 
animate(logging_dict, dataLabels, f"results/{dataset.name}_ani.gif", fast=True, plot_ee=True)